In [1]:
import os
import glob
import inro.modeller as m
import inro.emme.matrix as _matrix
import numpy as np
import pandas as pd
mm = m.Modeller()    
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
eb = mm.emmebank

In [2]:
scen_bau = 404

# scenario_2011 = 101
# scenario_2016 = 109
# scenario_2041 = 114

# ntwk_2011 = eb.scenario(scenario_2011).get_network()
# ntwk_2016 = eb.scenario(scenario_2016).get_network()
# ntwk_2041 = eb.scenario(scenario_2041).get_network()
            

In [3]:
### IMPORT ALL MATRICES IN SUBFOLDERS ###
def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)
def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"

In [4]:
#Matrix naming for inputs

# peak period - 7 hrs
sov_demand_pk = "TRIPS SOV peak"
hov2_demand_pk = "TRIPS HOV2 peak"
hov3_demand_pk = "TRIPS HOV3 peak"
bike_demand_pk = "TRIPS BIKE peak"
walk_demand_pk = "TRIPS WALK peak"
transit_demand_pk = "TRIPS TOTAL TRANSIT peak"

sov_demand_offpk = "TRIPS SOV offpeak"
hov2_demand_offpk = "TRIPS HOV2 offpeak"
hov3_demand_offpk = "TRIPS HOV3 offpeak"
bike_demand_offpk = "TRIPS BIKE offpeak"
walk_demand_offpk = "TRIPS WALK offpeak"
transit_demand_offpk = "TRIPS TOTAL TRANSIT offpeak"

#Truck matrices
truck_l_demand_am = "am_light_2051_20191227"
truck_m_demand_am = "am_medium_2051_20191227"
truck_h_demand_am = "am_heavy_2051_20191227"
truck_l_demand_md = "md_light_2051_20191227"
truck_m_demand_md = "md_medium_2051_20191227"
truck_h_demand_md = "md_heavy_2051_20191227"

# distance
sov_dist_am = "skim.am_peak.auto.sov.dist"
sov_dist_md = "skim.midday.auto.sov.dist"

#Below for road pricing run
# sov_dist_am = "skim.am_peak.auto.sovt.dist"
# sov_dist_md = "skim.midday.auto.sovt.dist"

# travel time
active_time = "skim.static.bike.time"

sov_time_am = "skim.am_peak.auto.sov.time"
hov2_time_am = "skim.am_peak.auto.hov2.time"
hov3_time_am = "skim.am_peak.auto.hov3.time"
sov_time_md = "skim.midday.auto.sov.time"
hov2_time_md = "skim.midday.auto.hov2.time"
hov3_time_md = "skim.midday.auto.hov3.time"

#Below for the road pricing run
# sov_time_am = "skim.am_peak.auto.sovt.time"
# hov2_time_am = "skim.am_peak.auto.hov2t.time"
# hov3_time_am = "skim.am_peak.auto.hov3t.time"
# sov_time_md = "skim.midday.auto.sovt.time"
# hov2_time_md = "skim.midday.auto.hov2t.time"
# hov3_time_md = "skim.midday.auto.hov3t.time"

#costs
sov_toll_am = "skim.am_peak.auto.sovt.toll_cost"
hov2_toll_am = "skim.am_peak.auto.hov2t.toll_cost"
hov3_toll_am = "skim.am_peak.auto.hov3t.toll_cost"

pk_gobus_fare = "skim.peak.transit.go_bus.fares"

In [5]:
#import all matrices from run output folder
hov2_oc = 2.2
hov3_oc = 3.3
#Delete all matrices
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_transit_fare = "mf180"
transit_aux_time = "mf6"
truck_cost = "mf190"

matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Medium List\BAU\BAU_Updated_APTG_2051_GGH_MTP_V106A3_CS222_20200727"
#scenario compatible with truck matrices
truck_scen = 404
scen_num = 404

mto_tt_skims = matrix_folder + "\\Peak minimum skims LVL3_PATH\\time_all"

clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_transit_fare,transit_aux_time,truck_cost],"FULL")
    
min_transit_time_peak = "mf140"

#minimum transit time stuff
create_matrix(matrix_id = min_transit_time_peak,
    matrix_name= "min.transit.time",
    matrix_description = "minimum.transit.time")

mat_list_mto = {"All Transit": ""}

for matrix_name in mat_list_mto.keys():
    mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = mat_list_mto[matrix_name],
              matrix_name= matrix_name.replace(" ","_"),
              matrix_description = matrix_name)

import_matrices_from_directory(mto_tt_skims, mat_list_mto, ".mdf", scen_bau)

spec = {
   "type": "MATRIX_CALCULATION",
   "result": min_transit_time_peak,
   "expression":  mat_list_mto["All Transit"],
     "constraint": {
"by_value": None,
"by_zone": None}
    }
matrix_calculator(spec)

spec2 = {
   "type": "MATRIX_CALCULATION",
   "result": min_transit_time_peak,
   "expression":  hdr_bau_peak,
     "constraint": {
"by_value": {
    "interval_min": 0,
    "interval_max": 0,
    "condition": "INCLUDE",
    "od_values":  min_transit_time_peak
},
"by_zone": None}
    }
matrix_calculator(spec2)

spec3 = {
   "type": "MATRIX_CALCULATION",
   "result": min_transit_time_peak,
   "expression":  hdr_bau_peak,
     "constraint": {
"by_value": {
    "interval_min": 0,
    "interval_max": 800,
    "condition": "EXCLUDE",
    "od_values":  min_transit_time_peak
},
"by_zone": None}
    }
matrix_calculator(spec3)

clear_matrices(eb,[min_transit_time_peak,hdr_bau_peak, hdr_bau_offpeak,local_transit_fare,transit_aux_time,truck_cost],"FULL")


def remove_zeros_spec(matrix_id):
    spec = {
    "type": "MATRIX_CALCULATION",
    "result": matrix_id,
    "expression": "1000",
    "constraint": {
        "by_value": {
            "od_values": matrix_id,
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE"
            }
        }
    }
    return spec

truck_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Long List\BAU\MTO_2051_GGH_MTP_5002_V106A3_HS032_20200117\Assignable matrices"

truck_list = {truck_h_demand_am: '',
 truck_l_demand_am: '',
 truck_m_demand_am: '',
 truck_h_demand_md: '',
 truck_l_demand_md: '',
 truck_m_demand_md: ''}

for matrix_name in truck_list.keys():
    truck_list[matrix_name] = eb.available_matrix_identifier('FULL')
    print matrix_name
    create_matrix(matrix_id = truck_list[matrix_name],
                  matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                  matrix_description = matrix_name)

import_matrices_from_directory(truck_folder, truck_list, ".mdf",404) 

#Matrices - can export without any other processing

matrix_list={sov_dist_am:"",
             sov_time_am:"",
             sov_time_md:"",
             hov2_time_am:"",
             active_time:"",
             sov_toll_am:"",
             hov2_toll_am:"",
            sov_demand_pk : "",
           hov2_demand_pk: "",
           hov3_demand_pk : "",
           sov_demand_offpk : "",
           hov2_demand_offpk : "",
            hov3_demand_offpk : "",
           bike_demand_pk : "",
           walk_demand_pk: "",
           bike_demand_offpk: "",
           walk_demand_offpk: "",
           transit_demand_pk: "",
           transit_demand_offpk: "",
            pk_gobus_fare: ""}
#              'skim.offpeak.transit.go_bus.fares':""} # replace with fare matrices we end up receiving
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    print matrix_name
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                  matrix_description = matrix_name)
import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",scen_bau) 

#Transit Access time - sum matrices - determine what we will receive from SAFO
# matrix_list={'skim.peak.transit.local_bus.total_wait': 71,
#              'skim.peak.transit.local_bus.access_walk': 76,
#              'skim.peak.transit.local_bus.egress_walk': 81}

# import_matrices_from_folder(matrix_folder, matrix_list,scen_num)


Imported matrix mf1
Finished Importing Matrices
am_heavy_2051_20191227
md_light_2051_20191227
am_light_2051_20191227
am_medium_2051_20191227
md_medium_2051_20191227
md_heavy_2051_20191227
Imported matrix mf1
Imported matrix mf3
Imported matrix mf4
Imported matrix mf7
Imported matrix mf2
Imported matrix mf5
Finished Importing Matrices
skim.peak.transit.go_bus.fares
skim.am_peak.auto.sov.dist
TRIPS HOV3 offpeak
TRIPS SOV peak
TRIPS WALK offpeak
TRIPS TOTAL TRANSIT offpeak
skim.midday.auto.sov.time
skim.am_peak.auto.hov2t.toll_cost
TRIPS HOV2 offpeak
TRIPS HOV3 peak
TRIPS SOV offpeak
skim.am_peak.auto.sovt.toll_cost
TRIPS TOTAL TRANSIT peak
TRIPS HOV2 peak
TRIPS BIKE peak
TRIPS WALK peak
TRIPS BIKE offpeak
skim.static.bike.time
skim.am_peak.auto.hov2.time
skim.am_peak.auto.sov.time
Imported matrix mf26
Imported matrix mf15
Imported matrix mf9
Imported matrix mf27
Imported matrix mf19
Imported matrix mf14
Imported matrix mf25
Imported matrix mf24
Imported matrix mf22
Imported matrix mf16
I

In [6]:
#Calculate demand matrices

hov_dem_pk = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = hov_dem_pk,
              matrix_name="trips.peak.hov",
            matrix_description = "trips.peak.hov")
exp = matrix_list[hov2_demand_pk] + "/" + str(hov2_oc) + "+" + matrix_list[hov3_demand_pk] + "/" + str(hov3_oc)

spec = {
    "type": "MATRIX_CALCULATION",
    "result": hov_dem_pk,
    "expression": exp
}
matrix_calculator(spec)

hov_dem_offpk = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = hov_dem_offpk,
              matrix_name="trips.offpeak.hov",
            matrix_description = "trips.offpeak.hov")   
exp = matrix_list[hov2_demand_offpk] + "/" + str(hov2_oc)+ "+" + matrix_list[hov3_demand_offpk]+ "/" + str(hov3_oc)
spec = {
    "type": "MATRIX_CALCULATION",
    "result": hov_dem_offpk,
    "expression": exp
}
matrix_calculator(spec)

active_dem_pk = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = active_dem_pk,
              matrix_name="trips.peak.active",
            matrix_description = "trips.peak.active")   
exp = matrix_list[bike_demand_pk] + "+" + matrix_list[walk_demand_pk]
spec = {
    "type": "MATRIX_CALCULATION",
    "result": active_dem_pk,
    "expression": exp
}
matrix_calculator(spec)

active_dem_offpk = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = active_dem_offpk,
              matrix_name="trips.offpeak.active",
            matrix_description = "trips.offpeak.active")

exp = matrix_list[bike_demand_offpk] + "+" + matrix_list[walk_demand_offpk]

spec = {
    "type": "MATRIX_CALCULATION",
    "result": active_dem_offpk,
    "expression": exp
}
matrix_calculator(spec)

print "Finished calculating demand"

Finished calculating demand


In [7]:
#Calculate overall Truck matrices for: light, medium, heavy

#Light Trucks
light_dem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = light_dem_truck,
              matrix_name="trips.peak.truck.light",
            matrix_description = "trips.peak.truck.light")
exp = "({}.max.{}').max.({}.max.{}')".format(truck_list[truck_l_demand_am],truck_list[truck_l_demand_am],
                                            truck_list[truck_l_demand_md],truck_list[truck_l_demand_md])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": light_dem_truck,
    "expression": exp
}
matrix_calculator(spec)

#Medium Trucks
med_dem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = med_dem_truck,
              matrix_name="trips.peak.truck.medium",
            matrix_description = "trips.peak.truck.medium")

exp = "({}.max.{}').max.({}.max.{}')".format(truck_list[truck_m_demand_am],truck_list[truck_m_demand_am],
                                            truck_list[truck_m_demand_md],truck_list[truck_m_demand_md])
spec = {
    "type": "MATRIX_CALCULATION",
    "result": med_dem_truck,
    "expression": exp
}
matrix_calculator(spec)

#Heavy Trucks
heavy_dem_truck = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = heavy_dem_truck,
              matrix_name="trips.peak.truck.heavy",
            matrix_description = "trips.peak.truck.heavy")

exp = "({}.max.{}').max.({}.max.{}')".format(truck_list[truck_h_demand_am],truck_list[truck_h_demand_am],
                                            truck_list[truck_h_demand_md],truck_list[truck_h_demand_md])

spec = {
    "type": "MATRIX_CALCULATION",
    "result": heavy_dem_truck,
    "expression": exp
}

matrix_calculator(spec)

print "Finished creating truck matrices"


Finished creating truck matrices


In [8]:
access_transit_time = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = access_transit_time,
              matrix_name="access_transit_time",
            matrix_description = "access_transit_time")
exp = "(" +transit_aux_time + ")/2" 
spec = {
    "type": "MATRIX_CALCULATION",
    "result": access_transit_time,
    "expression": exp
}
matrix_calculator(spec)

{'average': 0.061607,
 'expression_type': 'FULL',
 'maximum': 1722.20996,
 'maximum_at': {'destination': 8885, 'origin': 8885},
 'minimum': 0.0,
 'minimum_at': {'destination': 1019, 'origin': 1001},
 'num_evaluations': 12780625,
 'result_type': 'FULL',
 'sum': 787375.75}

In [9]:
#Calculate bi-drectional peak travel time
new_matrix = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = new_matrix,
              matrix_name="temp.matrix",
            matrix_description = "temp.matrix")
mats =[sov_time_am,sov_time_md,hov2_time_am,min_transit_time_peak, access_transit_time]

for matrix in mats:
    if matrix == min_transit_time_peak:
        mat_id = min_transit_time_peak
    elif matrix == access_transit_time:
        mat_id = access_transit_time
    else:
        mat_id = matrix_list[matrix]
        
    spec = {
        "type": "MATRIX_CALCULATION",
        "result": new_matrix,
        "expression": "(" + mat_id+ ".max." + mat_id + "')"
    }
    matrix_calculator(spec)
    spec = {
        "type": "MATRIX_CALCULATION",
        "result": mat_id,
        "expression": new_matrix
    }
    matrix_calculator(spec)
eb.delete_matrix(new_matrix)
print "Finished calculating bi-directional peak travel time"

Finished calculating bi-directional peak travel time


In [10]:
avg_transit_cost = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = avg_transit_cost,
              matrix_name="transit_cost_avg",
            matrix_description = "transit_cost_avg")
exp = "(" + local_transit_fare + "+" +  matrix_list[pk_gobus_fare] + ")/2" 

spec = {
    "type": "MATRIX_CALCULATION",
    "result": avg_transit_cost,
    "expression": exp
}
matrix_calculator(spec)

{'average': 8.261183,
 'expression_type': 'FULL',
 'maximum': 23.002157,
 'maximum_at': {'destination': 9137, 'origin': 7001},
 'minimum': 0.0,
 'minimum_at': {'destination': 9872, 'origin': 1001},
 'num_evaluations': 12780625,
 'result_type': 'FULL',
 'sum': 105583080.0}

In [17]:
output_cols = {
             "sov_dist":matrix_list[sov_dist_am],
             "sov_time":matrix_list[sov_time_am],
             "ff_time":matrix_list[sov_time_md],
             "hov_time":matrix_list[hov2_time_am],
             "active_time":matrix_list[active_time],
             "sov_cost":matrix_list[sov_toll_am],
             "hov_cost":matrix_list[hov2_toll_am],
             "transit_cost": avg_transit_cost,
             "light_truck_dem":light_dem_truck,
             "med_truck_dem":med_dem_truck,
             "heavy_truck_dem":heavy_dem_truck,
             "transit_access_time": access_transit_time,
             "transit_time": min_transit_time_peak,
             "hov_dem_pk": hov_dem_pk,
    "hov_dem_offpk":hov_dem_offpk,
    "active_dem_pk":active_dem_pk,
    "active_dem_offpk":active_dem_offpk,
    "sov_dem_pk":matrix_list[sov_demand_pk],
    "sov_dem_offpk":matrix_list[sov_demand_offpk],
    "transit_dem_pk":matrix_list[transit_demand_pk],
    "transit_dem_offpk":matrix_list[transit_demand_offpk]}#,
    #"truck_cost": truck_cost}

In [18]:
first = True
indices = eb.matrix(matrix_list[sov_demand_pk]).get_data(scen_bau).indices
for col in output_cols:
    mat_np = eb.matrix(output_cols[col]).get_numpy_data(scen_bau)
    if first:
        df_full = pd.DataFrame(mat_np,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "O",
                                                                                           "level_1": "D",
                                                                                          0: col})
        first = False
    else:
        df = pd.DataFrame(mat_np,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "O",
                                                                                           "level_1": "D",
                                                                                          0: col})
        df_full = df_full.merge(df,on = ["O","D"])
        

In [16]:
df_full.columns

Index([u'O', u'D', u'active_time', u'transit_access_time', u'light_truck_dem', u'sov_dem_offpk', u'ff_time', u'hov_dem_offpk', u'hov_cost', u'transit_dem_offpk', u'active_dem_offpk', u'sov_dist', u'transit_dem_pk', u'active_dem_pk', u'hov_time', u'sov_cost', u'hov_dem_pk', u'sov_dem_pk', u'transit_time', u'sov_time', u'med_truck_dem', u'transit_cost'], dtype='object')

In [25]:
df_full.sov_cost = df_full.sov_cost/100
df_full.hov_cost = df_full.hov_cost/100
df_full["dem_truck"] = df_full.light_truck_dem + df_full.med_truck_dem + df_full.heavy_truck_dem
df_full["dem_pk"] = df_full.sov_dem_pk + df_full.hov_dem_pk+df_full.transit_dem_pk+df_full.active_dem_pk

In [18]:
# demand_weighted = {u'active_time':u'active_dem_pk',u'ff_time':u'sov_dem_pk',u'sov_time':u'sov_dem_pk',
#                    u'hov_cost':u'hov_dem_pk',u'sov_dist':u'sov_dem_pk',u'hov_time':u'sov_dem_pk',
#                    u'sov_cost':u'sov_dem_pk'}

demand_weighted = {u'active_time':"dem_pk",u'ff_time':"dem_pk",u'sov_time':"dem_pk",
                   u'hov_cost':"dem_pk",u'sov_dist':"dem_pk",u'hov_time':"dem_pk",
                   u'sov_cost':"dem_pk",u'transit_cost':"dem_pk",u'transit_time':"dem_pk",
                  "truck_cost":"dem_truck","transit_access_time":"dem_pk"}

In [19]:
for met in demand_weighted.keys():
    df_full[met + "_dem_weight"] = df_full[met]*df_full[demand_weighted[met]]

In [20]:
pth = r"C:\Users\bcrane\Documents\Sketch Modelling"
megazones_df = pd.read_csv(os.path.join(pth,"TAZ_SketchModelZones_20200911.csv"))

In [26]:
df_internal = df_full[(df_full.O <= 9331) & (df_full.D <= 9331)]

In [22]:
df_internal = df_internal.merge(megazones_df, left_on = "O", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_O"})
df_internal = df_internal.merge(megazones_df, left_on = "D", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_D"})

In [36]:
df_internal.columns

Index([u'O', u'D', u'active_time', u'active_dem_offpk', u'transit_dem_pk', u'ff_time', u'sov_time', u'hov_cost', u'active_dem_pk', u'transit_dem_offpk', u'med_truck_dem', u'hov_dem_offpk', u'sov_dem_offpk', u'light_truck_dem', u'sov_dist', u'hov_time', u'sov_cost', u'heavy_truck_dem', u'hov_dem_pk', u'sov_dem_pk', u'dem_truck', u'active_time_dem_weight', u'ff_time_dem_weight', u'sov_time_dem_weight', u'hov_cost_dem_weight', u'sov_dist_dem_weight', u'hov_time_dem_weight', u'sov_cost_dem_weight', u'dem_pk', u'Megazone_O', u'Megazone_D'], dtype='object')

In [25]:
def average(list):
    return sum(list)/len(list)

In [28]:
def median(list):
    return (max(list) + min(list))/2

In [23]:
pivot_columns = [u'active_time_dem_weight',u'ff_time_dem_weight',u'sov_time_dem_weight',u'sov_cost_dem_weight',
                u'hov_cost_dem_weight',u'sov_dist_dem_weight',u'hov_time_dem_weight',
                u'active_dem_offpk', u'transit_dem_pk', u'active_dem_pk', u'transit_dem_offpk',
                u'hov_dem_offpk', u'sov_dem_offpk', u'dem_truck', u'hov_dem_pk', u'sov_dem_pk',"dem_pk",
                u'transit_cost_dem_weight',u'transit_time_dem_weight',"truck_cost_dem_weight","transit_access_time_dem_weight"]
megazone_df = pd.pivot_table(df_internal,values = pivot_columns, index = ["Megazone_O","Megazone_D"],aggfunc = sum).reset_index()

In [24]:
for met in demand_weighted.keys():
    megazone_df[met] = megazone_df[met + "_dem_weight"] / megazone_df[demand_weighted[met]]
    megazone_df = megazone_df.drop(labels = met + "_dem_weight", axis = 1)

In [25]:
megazone_df.fillna(0).to_csv(os.path.join(pth, "2051BAU_MegazoneOutputs_20201104.csv"))

# External Calculations Below

In [27]:
pth = r"C:\Users\bcrane\Documents\Sketch Modelling"
megazones_df = pd.read_csv(os.path.join(pth,"TAZ_SketchModelZones_20201110.csv"))
df_int_ext = df_full[((df_full.O <= 9331) | (df_full.O == 9401)| (df_full.O == 9411) | (df_full.O == 9406) | (df_full.O == 9501)) & ((df_full.D <= 9331) | (df_full.D == 9401)| (df_full.D == 9411) | (df_full.D == 9406) | (df_full.D == 9501))]
df_int_ext = df_int_ext.merge(megazones_df, left_on = "O", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_O"})
df_int_ext = df_int_ext.merge(megazones_df, left_on = "D", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_D"})

In [28]:
df_int_ext.columns

Index([u'O', u'D', u'active_time', u'active_dem_offpk', u'transit_dem_pk', u'ff_time', u'sov_time', u'hov_cost', u'active_dem_pk', u'transit_time', u'transit_dem_offpk', u'med_truck_dem', u'hov_dem_offpk', u'transit_cost', u'light_truck_dem', u'sov_dem_offpk', u'sov_dist', u'transit_access_time', u'hov_time', u'sov_cost', u'heavy_truck_dem', u'hov_dem_pk', u'sov_dem_pk', u'dem_truck', u'dem_pk', u'Megazone_O', u'Megazone_D'], dtype='object')

In [29]:
megazone_df = pd.pivot_table(df_int_ext,values = "dem_pk", index = ["Megazone_O","Megazone_D"],aggfunc = sum).reset_index()

In [32]:
megazone_df[(megazone_df.Megazone_O == "Extl West") | (megazone_df.Megazone_O == "Extl North") | (megazone_df.Megazone_O == "Extl South")| (megazone_df.Megazone_O == "Extl East")|(megazone_df.Megazone_D == "Extl West") | (megazone_df.Megazone_D == "Extl North") | (megazone_df.Megazone_D == "Extl South")| (megazone_df.Megazone_D == "Extl East")]["dem_pk"].sum()

0.0